Okay, to start off, I'm not a computer vision specialist, as I worked more with text.

BUT, I'll share and try to mix everything I find that seems cool or good or full of potential

Hope you enjoy ❤

# Shoutouts:
Many thanks to the authors of these kernels (a.k.a. I'm basing this on these guys):
* Model: https://www.kaggle.com/corochann/vinbigdata-detectron2-train/data#Dataset-preparation
* Data processing and resizing: https://www.kaggle.com/xhlulu/vinbigdata-process-and-resize-original-ratio / https://www.kaggle.com/xhlulu/vinbigdata-process-and-resize-to-image
* Data reading: https://www.kaggle.com/raddar/convert-dicom-to-np-array-the-correct-way
* Prediction Kernel: https://www.kaggle.com/corochann/vinbigdata-detectron2-prediction

# Imports

## Installations

In [ ]:
!nvcc --version

In [ ]:
!nvidia-smi

In [ ]:
!pip uninstall detectron2 -y

In [ ]:
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu102/torch1.7/index.html

In [ ]:
######################################## Python/General #######################################
import os
from PIL import Image # it's image related, but it's kinda default to python, so it stays here
from tqdm import tqdm
import pickle
import pathlib
from pathlib import Path
import typing
from typing import Optional, Any, Union, Dict, List
from tqdm import tqdm
import yaml
import contextlib
import copy
import io
import itertools
import json
import logging
from collections import OrderedDict
from tabulate import tabulate
import dataclasses
from dataclasses import dataclass
from typing import Dict
import argparse
import random
import sys
from distutils.util import strtobool
import math

######################################## Data science #########################################
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt

%matplotlib inline

######################################## Image related ########################################
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut
import cv2

# Dataset
for dirname, _, filenames in os.walk('/kaggle/input'):
    print(dirname)  
    
######################################## Model related ########################################    
import torch
import torch.nn as nn
from detectron2.structures import BoxMode
import detectron2.data.transforms as T
from detectron2.data import detection_utils as utils
from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.engine import DefaultPredictor, DefaultTrainer, launch
from detectron2.evaluation import COCOEvaluator
from detectron2.structures import BoxMode
from detectron2.utils.logger import setup_logger
from detectron2.utils.visualizer import Visualizer
from detectron2.utils.visualizer import ColorMode, Visualizer

print(torch.__version__)

# Reading images

In [ ]:
def read_xray(path, voi_lut = True, fix_monochrome = True):
    dicom = pydicom.read_file(path)
    
    # VOI LUT (if available by DICOM device) is used to transform raw DICOM data to "human-friendly" view
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array
        
    # Depending on this value, X-ray may look inverted - fix that
    if fix_monochrome and dicom.PhotometricInterpretation == 'MONOCHROME1"':
        data = np.amax(data) - data
        
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
    
    return data

In [ ]:
img = read_xray('../input/vinbigdata-chest-xray-abnormalities-detection/train/000434271f63a053c4128a0ba6352c7f.dicom')
plt.figure(figsize = (12, 12))
plt.imshow(img, 'gray')

# Resizing

In [ ]:
#def resize(array, size, keep_ratio=False, resample=Image.LANCZOS):
#    im = Image.fromarray(array)
    
#    if keep_ratio:
#        im.thumbnail((size, size), resample)
#    else:
#        im = im.resize((size, size), resample)
        
#    return im

In [ ]:
#image_id = []
#dim0 = []
#dim1 = []

#for split in ['train', 'test']:
#    load_dir = f'../input/vinbigdata-chest-xray-abnormalities-detection/{split}/'
#    save_dir = f'/kaggle/tmp/{split}/'
    
#    os.makedirs(save_dir, exist_ok=True)

#    for file in tqdm(os.listdir(load_dir)):
#        # set keep_ratio=True to have originicar aspect ratio
#        xray = read_xray(load_dir + file)
#        im = resize(xray, size=512, keep_ratio=True)
#        im.save(save_dir + file.replace('dicom', 'png'))
        
#        if split == 'train':
#           image_id.append(file.replace('.dicom', ''))
#            dim0.append(xray.shape[0])
#            dim1.append(xray.shape[1])

# Inference methods

In [ ]:
def get_vinbigdata_dicts(imgdir, train_df, train_data_type='original', 
                         use_cache=True, debug=True, 
                         np_ndarr_target_indices=None, use_class14=False):
    
    debug_str = f'_debug{int(debug)}'
    
    train_data_type_str = f'_{train_data_type}'
    
    class_14_str = f'_14_class{int(use_class_14)}'
    
    cache_path = Path('.') / f'dataset_dicts_cache{train_data_type_str}{class_14_str}{debug_str}.pkl'
    
    if not use_cache or not cache_path.exists():
        print('Creating data...')
        train_meta = pd.read_csv(imgdir / 'train_meta.csv')
        
        if debug:
            train_df = train_df.iloc[:500]
        
        image_id = train_df.loc[0, 'image_id']
        image_path = str(imgdir / 'train' / f'{image_id}.png')
        image = cv2.imread(image_path)
        img_height, img_width, ch = image.shape
        
        print(f'image shape: {image.shape}')
        
        dataset_dicts = []
        
        for index, train_meta_row in tqdm(train_meta.iterrows(), total=len(traqin_meta)):
            
            image_id, height, width = train_meta_row.values
            filename = str(imgdir / 'train' / f'{image_id}.png')
            
            record = {}
            
            record['file_name'] = filename
            record['image_id']  = image_id
            record['height']    = img_height
            record['width']     = img_width
            
            objects = []

            for index2, row in train_df.query('image_id == @image_id').iterrows():
                class_id = row['class_id']
                if class_id == 14:
                    if use_class_14:
                        bbox_resized = [0, 0, img_width, img_height]
                        
                        obj = {}
                        
                        obj['bbox']        = bbox_resized
                        obj['bbox_mode']   = BoxMode.XYXY_ABS
                        obj['category_id'] = class_id
                        
                        objs.append(obj)
                    else:
                        pass
                    
                else:
                    h_ratio = img_height / heigth
                    w_ratio = img_width / width
                    bbox_resized = [float(row['x_min']) * w_ratio,
                                    float(row['y_min']) * h_ratio,
                                    float(row['x_max']) * w_ratio,
                                    float(row['y_max']) * h_ratio]
                    
                    obj = {}
                    
                    obj['bbox'] = bbox_resized
                    obj['bbox_mode'] = BoxMode.XYXY_ABS
                    obj['category_id'] = class_id
                    
                    objs.append(obj)
                    
            record['annotations'] = objs
            dataset_dict.append(record)
            
        with open(cache_path, mode='wb') as f:
            pickle.dump(dataset_dicts, f)
            
    print(f'Load from cache {cache_path}')
    with open(cache_path, mode='rb') as f:
        dataset_dicts = pickle.load(f)
    
    if target_indices is not None:
        dataset_dicts = [dataset_dicts[i] for i in target_indices]
    return dataset_dicts

In [ ]:
def get_vinbigdata_dicts_test(Path, test_df, use_cache=True, debug=True):
    debug_str = f'_debug{int(debug)}'
    cache_path = pathlib.Path('.') / f'dataset_dicts_cache_test{debug_str}.pkl'
    
    if not use_cache or not cache_path.exists():
        print("Creating data...")
        
        if debug:
            test_df = test_df.iloc[:500]
            
        image_id = test_df.loc[0, 'image_id']
        image_path = str(imgdir / 'test' / f'{image_id}.png')
        image = cv2.imread(image_path)
        img_height, img_width, ch = image.shape
        
        print(f'image shape: {image.shape}')
        
        dataset_dicts = []
        
        for index, test_meta_row in tqdm(test_meta.iterrows(), total = len(test_meta)):
            image_id, img_heigth, img_width = test_meta_row.values
            filename = str(imgdir / 'test' / f'{image_id}.png')
            
            record = {}
            
            record['file_name'] = filename
            record['image_id']  = image_id
            record['height']    = img_heigth
            record['width']     = img_width
            
            dataset_dicts.append(record)
            
        with open(cache_path, mode="wb") as f:
            pickle.dump(dataset_dicts, f)

    print(f"Load from cache {cache_path}")
    
    with open(cache_path, mode="rb") as f:
        dataset_dicts = pickle.load(f)
        
    return dataset_dicts

In [ ]:
def save_yaml(filepath: Union[str, Path], content: Any, width: int=120):
    with open(filepath, "w") as f:
        yaml.dump(content, f, width=width)


def load_yaml(filepath: Union[str, Path]) -> Any:
    with open(filepath, "r") as f:
        content = yaml.full_load(f)
        
    return content

In [ ]:
def format_pred(labels: np.ndarray, boxes: np.ndarray, scores: np.ndarray) -> str:
    pred_strings = []
    for label, score, bbox in zip(labels, scores, boxes):
        xmin, ymin, xmax, ymax = bbox.astype(np.int64)
        pred_strings.append(f"{label} {score} {xmin} {ymin} {xmax} {ymax}")
        
    return " ".join(pred_strings)


def predict_batch(predictor: DefaultPredictor, im_list) -> typing.List:
    with torch.no_grad():  # https://github.com/sphinx-doc/sphinx/issues/4258
        inputs_list = []
        for original_image in im_list:
            # Apply pre-processing to image.
            if predictor.input_format == "RGB":
                # whether the model expects BGR inputs or RGB
                original_image = original_image[:, :, ::-1]
            height, width = original_image.shape[:2]
            # Do not apply original augmentation, which is resize.
            # image = predictor.aug.get_transform(original_image).apply_image(original_image)
            image = original_image
            image = torch.as_tensor(image.astype("float32").transpose(2, 0, 1))
            inputs = {"image": image, "height": height, "width": width}
            inputs_list.append(inputs)
        predictions = predictor.model(inputs_list)
        return predictions

In [ ]:
region_classes = [
    'Aortic enlargement',
    'Atelectasis',
    'Calcification',
    'Cardiomegaly',
    'Consolidation',
    'ILD',
    'Infiltration',
    'Lung opacity',
    'Nodule/Mass',
    'Other lesion'
    'Pleural effusion',
    'Pleural thickening',
    'Pneumothorax',
    'Pulmonary fibrosis'
]

category_name_to_id = {class_name: index for index, class_name in enumerate(region_classes)}

In [ ]:
class Config:
    # General
    debug = True
    outdir = 'results/det'
    
    # Data config
    #vinbigdata-chest-xray-resized-png-256x256
    imgdir_name = 'vinbigdata'
    split_mode = 'all_train'  # all_train or valid20
    seed = 111
    train_data_type = 'original'  # original or wbf
    use_class14 = False
        
    # Training config
    iter = 10000
    ims_per_batch = 2  # images per batch, this corresponds to "total batch size"
    num_workers = 4
    lr_scheduler_name = 'WarmupMultiStepLR'  # WarmupMultiStepLR (default) or WarmupCosineLR
    base_lr = 0.00025
    roi_batch_size_per_image = 512
    eval_period = 10000
    aug_kwargs = dataclasses.field(default_factory=lambda: {})

    def update(self, param_dict) -> 'Config':
        # Overwrite by `param_dict`
        for key, value in param_dict.items():
            if not hasattr(self, key):
                raise ValueError(f'[ERROR] Unexpected key for config = {key}')
            setattr(self, key, value)
        return self

In [ ]:
inputdir = Path('/kaggle/input')
traineddir = inputdir / 'vinbigdata-r50fpn3x-512px'

configs = Config().update(load_yaml(str(traineddir/'flags.yaml'))) # TODO: change to configs.yaml when there's time to gen 
print('configs', configs)
debug = configs.debug
outdir = Path(configs.outdir)
os.makedirs(str(outdir), exist_ok=True)

# --- Read data ---
datadir = inputdir / 'vinbigdata-chest-xray-abnormalities-detection'
if configs.imgdir_name == 'vinbigdata2':
    #vinbigdata
    imgdir = inputdir/ 'vinbigdata-chest-xray-resized-png-256x256'
else:
    imgdir = inputdir / configs.imgdir_name

# Read in the data CSV files
# train = pd.read_csv(datadir / "train.csv")
test_meta = pd.read_csv(inputdir / 'vinbigdata-testmeta' / 'test_meta.csv')
sample_submission = pd.read_csv(datadir / 'sample_submission.csv')

In [ ]:
cfg = get_cfg()
original_output_dir = cfg.OUTPUT_DIR
cfg.OUTPUT_DIR = str(outdir)
print(f'cfg.OUTPUT_DIR {original_output_dir} -> {cfg.OUTPUT_DIR}')

cfg.merge_from_file(model_zoo.get_config_file('COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml'))
cfg.DATASETS.TRAIN = ('vinbigdata_train',)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url('COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml')
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = configs.base_lr  # pick a good LR
cfg.SOLVER.MAX_ITER = configs.iter
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = configs.roi_batch_size_per_image
cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(region_classes)

### --- Inference & Evaluation ---
# Inference should use the config with parameters that are used in training
# cfg now already contains everything we've set previously. We changed it a little bit for inference:
# path to the model we just trained
cfg.MODEL.WEIGHTS = str(traineddir/'model_final.pth')
print('Original thresh', cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST)  # 0.05
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.13  # set a custom testing threshold
print('Changed  thresh', cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST)
predictor = DefaultPredictor(cfg)

#DatasetCatalog.register(
#    'vinbigdata_test', lambda: get_vinbigdata_dicts_test(imgdir, test_meta, debug=debug)
#)
MetadataCatalog.get('vinbigdata_test').set(thing_classes=region_classes)
metadata = MetadataCatalog.get('vinbigdata_test')
dataset_dicts = get_vinbigdata_dicts_test(imgdir, test_meta, debug=debug)

if debug:
    dataset_dicts = dataset_dicts[:100]

results_list = []
index = 0
batch_size = 4

for i in tqdm(range(math.ceil(len(dataset_dicts) / batch_size))):
    inds = list(range(batch_size * i, min(batch_size * (i + 1), len(dataset_dicts))))
    dataset_dicts_batch = [dataset_dicts[i] for i in inds]
    im_list = [cv2.imread(d['file_name']) for d in dataset_dicts_batch]
    outputs_list = predict_batch(predictor, im_list)

    for im, outputs, d in zip(im_list, outputs_list, dataset_dicts_batch):
        resized_height, resized_width, ch = im.shape
        # outputs = predictor(im)
        if index < 5:
            # format is documented at https://detectron2.readthedocs.io/tutorials/models.html#model-output-format
            v = Visualizer(
                im[:, :, ::-1],
                metadata=metadata,
                scale=0.5,
                instance_mode=ColorMode.IMAGE_BW
                # remove the colors of unsegmented pixels. This option is only available for segmentation models
            )
            out = v.draw_instance_predictions(outputs['instances'].to('cpu'))
            # cv2_imshow(out.get_image()[:, :, ::-1])
            cv2.imwrite(str(outdir / f'pred_{index}.jpg'), out.get_image()[:, :, ::-1])

        image_id, dim0, dim1 = test_meta.iloc[index].values

        instances = outputs['instances']
        if len(instances) == 0:
            # No finding, let's set 14 1 0 0 1 1x.
            result = {'image_id': image_id, 'PredictionString': '14 1.0 0 0 1 1'}
        else:
            # Find some bbox...
            # print(f"index={index}, find {len(instances)} bbox.")
            fields: Dict[str, Any] = instances.get_fields()
            pred_classes = fields['pred_classes']  # (n_boxes,)
            pred_scores = fields['scores']
            # shape (n_boxes, 4). (xmin, ymin, xmax, ymax)
            pred_boxes = fields['pred_boxes'].tensor

            h_ratio = dim0 / resized_height
            w_ratio = dim1 / resized_width
            pred_boxes[:, [0, 2]] *= w_ratio
            pred_boxes[:, [1, 3]] *= h_ratio

            pred_classes_array = pred_classes.cpu().numpy()
            pred_boxes_array = pred_boxes.cpu().numpy()
            pred_scores_array = pred_scores.cpu().numpy()

            result = {
                'image_id': image_id,
                'PredictionString': format_pred(
                    pred_classes_array, pred_boxes_array, pred_scores_array
                ),
            }
        results_list.append(result)
        index += 1

In [ ]:
submission_det = pd.DataFrame(results_list, columns=['image_id', 'PredictionString'])
submission_det.to_csv(outdir/"submission.csv", index=False)
submission_det